In [17]:
import pandas as pd
import numpy as np

In [18]:
# Unificar tweets
tweets_k = pd.read_pickle('../datasets/TweetsDataset_K.pkl')[['screen_name','text','created_at']]
tweets_k['grupo'] = 'K'
tweets_m = pd.read_pickle('../datasets/TweetsDataset_M.pkl')[['screen_name','text','created_at']]
tweets_m['grupo'] = 'M'
tweets = pd.concat([tweets_k,tweets_m])

In [19]:
# Resumen del DataSet
print('K-Tweets_count:',tweets_k.screen_name.count())
print('K-Users_count:',tweets_k.screen_name.nunique())
print('M-Tweets_count:',tweets_m.screen_name.count())
print('M-Users_count:',tweets_m.screen_name.nunique())

K-Tweets_count: 252388
K-Users_count: 386
M-Tweets_count: 262108
M-Users_count: 411


### Sentiment Analysis

In [20]:
# Importar e Instanciar Sentiment Classifier
# Gracias y créðitos a https://github.com/aylliote/senti-py
from classifier import *
clf = SentimentClassifier()

In [21]:
# Función para iterar por la lista, predecir y anexar predicción como columna nueva y entregar dataframe resultante
def calc_sent(df):
    lista = []
    for index,row in df.iterrows():
        lista.append([row.grupo,row.created_at,row.screen_name,clf.predict(row.text),row.text])
    return pd.DataFrame(lista,columns=['grupo','created_at','screen_name','sent_score','text'])

In [22]:
# Función para paralelizar el bucle de la función calc_sent
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
# Correr cálculo de sent_scores para todo el dataframe
from multiprocessing import Pool
tweets = parallelize_dataframe(tweets,calc_sent,n_cores=28)

In [ ]:
# Agregar media por usuario de Sent_Score
tweets = tweets.join(tweets[['screen_name','sent_score']].groupby('screen_name').mean(),on='screen_name',rsuffix='_mean')
# Convertir a categórica
tweets['user_type'] = pd.cut(tweets['sent_score_mean'], [0,0.2, 0.4, 0.6, 0.8, 1] , labels=["hater", "negative", "neutral","positive","lover"])

In [ ]:
# Agregar columna con zscore del score de cada usuario
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
tweets['sent_zscore_mean'] = scaler.fit_transform(tweets[['sent_score_mean']])

In [ ]:
# Drop a un archivo excel para procesar
import openpyxl
tweets.to_excel('../datasets/tweets_M_K.xlsx',index=False)